In [266]:
import pandas as pd
import os
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')

In [145]:
DAYS_IN_MONTH = 31  #assuming equal number of days in each month
TODAY = datetime.datetime(2023, 3, 26)

### Convert the csv file directly to excel since reading the csv file raises some errors

In [55]:
csv_file = 'historic_transactions.csv'
excel_filename = os.path.splitext(csv_file)[0]+'.xlsx'
to_excel = os.rename(csv_file, excel_filename)

In [57]:
db = pd.read_excel('historic_transactions.xlsx')

In [58]:
db.sample(3)

,transaction_date,transaction_id,transaction_amount
912,2023-02-12,8IJ9cRaxuaa1wMYs70bS,1300
590,2023-01-16,m4IWRQw9mhQjqhcj8YkP,1500
894,2023-02-22,IsB9VPbns6jrLN4Sujt2,750


In [59]:
db.shape

(1109, 3)

In [60]:
db.dtypes

transaction_date      datetime64[ns]
transaction_id                object
transaction_amount             int64
dtype: object

In [64]:
db['transaction_date'].min()

Timestamp('2022-09-03 00:00:00')

### Calculate the number of days left until the end of March as `n`

In [85]:
# month_name -- name of the month gotten from the datetime..

db['transaction_date'] = pd.to_datetime(db['transaction_date'])

In [86]:
db['month_name'] = db['transaction_date'].dt.strftime('%B')

In [87]:
db.head()

,transaction_date,transaction_id,transaction_amount,month_name
0,2022-09-17,WeKuSlJdWG0OfbkteXN5,1000,September
1,2022-09-16,bypPVdMn7iydzNwsYBZl,2000,September
2,2022-09-15,k7ZEkPT3nV078GbXdjX9,2000,September
3,2022-09-22,KSKjgWG5f3vFj7mCh6Sf,1500,September
4,2022-09-16,6NkQBzah4sUEWaO31gQv,1000,September


### Get the subset which contains only the month of March

In [89]:
march_df = db.query("month_name == 'March'")

In [90]:
march_df

,transaction_date,transaction_id,transaction_amount,month_name
641,2023-03-20,IAJlw0mHSeTni5fsMhPS,600,March
642,2023-03-12,Glyn0oc4p4z1dOVJtS3Z,2600,March
643,2023-03-19,8eSwFg5jKm3zlpZpfILr,3478,March
644,2023-03-12,jxry1UtcRUwndjUpcULl,1900,March
645,2023-03-16,krJoJ9uYSADaV9Bq7Ayk,2300,March
...,...,...,...,...
845,2023-03-24,qTWHVZ4hWMrblQzjm68h,709,March
846,2023-03-14,ymVMsNy4PUzv4Ullq3H1,2500,March
847,2023-03-17,LhMEw1oa9BNpk9esEYf5,2200,March
848,2023-03-12,lgbAoSnNaJA3hjn9KOab,2200,March


In [146]:
# Get the number of days left for March to end;

n = DAYS_IN_MONTH - TODAY.day
n

5

### Start calculating the volume for each month starting from September 2022 to February 2023

In [139]:
sept_to_feb = db.query("month_name != 'March'")

In [141]:
sept_to_feb.shape

(900, 4)

In [153]:
volume_of_months = sept_to_feb.groupby('month_name').sum('transaction_amount').reset_index()
volume_of_months.rename(columns={'transaction_amount': 'total_volume'}, inplace=True)

In [154]:
volume_of_months['volume_run_rate'] = volume_of_months['total_volume']/DAYS_IN_MONTH

In [155]:
volume_of_months

,month_name,total_volume,volume_run_rate
0,December,289850,9350.000000
1,February,414240,13362.580645
2,January,294150,9488.709677
3,November,176800,5703.225806
4,October,160900,5190.322581
5,September,132350,4269.354839


### Get the volume run rate of each month `n` days before the end of that month

In [157]:
# get the subset of the data that falls in the 5 days to the end of month.. thus from day 1-26 of every month.

n_days_before_monthend = db[(db['transaction_date'].dt.day >=1) & (db['transaction_date'].dt.day <=26)]

In [158]:
n_days_before_monthend.head()

,transaction_date,transaction_id,transaction_amount,month_name
0,2022-09-17,WeKuSlJdWG0OfbkteXN5,1000,September
1,2022-09-16,bypPVdMn7iydzNwsYBZl,2000,September
2,2022-09-15,k7ZEkPT3nV078GbXdjX9,2000,September
3,2022-09-22,KSKjgWG5f3vFj7mCh6Sf,1500,September
4,2022-09-16,6NkQBzah4sUEWaO31gQv,1000,September


In [159]:
n_days_before_monthend.shape

(1090, 4)

### Get the volume run rate for each month 5 days before the end of the month

In [168]:
FIVE_DAYS_TO_END = DAYS_IN_MONTH - n
FIVE_DAYS_TO_END

26

In [192]:
n_days_volume_run_rate = n_days_before_monthend.groupby('month_name').sum('transaction_amount').reset_index()

In [194]:
n_days_volume_run_rate.rename(columns={'transaction_amount': 'total_volume'}, inplace=True)

In [195]:
n_days_volume_run_rate['volume_run_rate'] = n_days_volume_run_rate['total_volume']/FIVE_DAYS_TO_END

In [196]:
n_days_volume_run_rate.dtypes

month_name          object
total_volume         int64
volume_run_rate    float64
dtype: object

In [197]:
n_days_volume_run_rate

,month_name,total_volume,volume_run_rate
0,December,287450,11055.769231
1,February,410340,15782.307692
2,January,285850,10994.230769
3,March,359463,13825.500000
4,November,176800,6800.000000
5,October,155600,5984.615385
6,September,127850,4917.307692


### Now we'll use the two new datasets to estimate the volume by the end of March

In [204]:
# Add number of days used to calculate the volume run rates..
# add column 'number_of_days' = 31 to the first dataset and
# add 26 for dataset that consist of 5 days to the end of month..

n_days_volume_run_rate['number_of_days'] = 26
volume_of_months['number_of_days'] = 31


In [205]:
# combine the two volume run rate datasets..

df = pd.concat([volume_of_months, n_days_volume_run_rate], axis=0)

In [206]:
df

,month_name,total_volume,volume_run_rate,number_of_days
0,December,289850,9350.000000,31
1,February,414240,13362.580645,31
2,January,294150,9488.709677,31
3,November,176800,5703.225806,31
4,October,160900,5190.322581,31
5,September,132350,4269.354839,31
0,December,287450,11055.769231,26
1,February,410340,15782.307692,26
2,January,285850,10994.230769,26
3,March,359463,13825.500000,26


In [212]:
# add another column known as month_number which represents the number of the month.
# Jan = 1, feb = 2 ......and so on.

month_mappings = {
    'January' : 1,
    'February' : 2,
    'March' : 3,
    'September' : 9,
    'October' : 10,
    'November' : 11,
    'December' : 12
}

df['month_number'] = df['month_name'].map(month_mappings)

In [213]:
df

,month_name,total_volume,volume_run_rate,number_of_days,month_number
0,December,289850,9350.000000,31,12
1,February,414240,13362.580645,31,2
2,January,294150,9488.709677,31,1
3,November,176800,5703.225806,31,11
4,October,160900,5190.322581,31,10
5,September,132350,4269.354839,31,9
0,December,287450,11055.769231,26,12
1,February,410340,15782.307692,26,2
2,January,285850,10994.230769,26,1
3,March,359463,13825.500000,26,3


In [214]:
df.columns

Index(['month_name', 'total_volume', 'volume_run_rate', 'number_of_days',
       'month_number'],
      dtype='object')

In [222]:
features = ['number_of_days', 'month_number']
target_1 = ['volume_run_rate']
target_2 = ['total_volume']

### Let's use a simple linear regression Or Random Forest regressor to estimate the volume run rate by the end of march.

In [217]:
# import estimators/predictors
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [256]:
estimator_1 =  RandomForestRegressor()
estimator_2 = RandomForestRegressor()

In [267]:
estimator_1.fit(df[features], df[target_1])

RandomForestRegressor()

In [268]:
estimator_2.fit(df[features], df[target_2])

RandomForestRegressor()

In [271]:
print(f"March volume run rate estimate is: {estimator_1.predict(np.array([31, 3]).reshape(1,-1))[0]}")

March volume run rate estimate is: 13226.089975186102


In [270]:
print(f"March total volume estimate is: {estimator_2.predict(np.array([31, 3]).reshape(1,-1))[0]}")

March total volume estimate is: 371610.6


## MySQL Query

SELECT
	request_id,
	customer_id,
	request_datetime,
	request_amount,
CASE
		
		WHEN RANK() OVER ( PARTITION BY customer_id ORDER BY request_datetime ASC )= 1 THEN
		1 ELSE 0 
	END AS first_request_flag 
FROM
	requests;